### p.310 여러 번 재사용되는 텐서

* 텐서 = 벡터와 행렬의 추상적 형태

In [1]:
import numpy as np

class Tensor(object):
    def __init__(self, data):
        self.data = np.array(data)
    def __add__(self, other):
        return Tensor(self.data + other.data)
    def __repr__(self):
        return str(self.data.__repr__())
    def __str__(self):
        return str(self.data.__str__())

In [2]:
x = Tensor([1, 2, 3, 4, 5])
print(x)

[1 2 3 4 5]


In [4]:
y = x+x
print(y)

[ 2  4  6  8 10]


In [6]:
class Tensor(object):
    def __init__(self, data, creators=None, creation_op=None):
        self.data = np.array(data)
        self.creation_op = creation_op
        self.creators = creators
        self.grad = None
        
    def backward(self, grad):
        self.grad = grad
        if(self.creation_op == "add"):
            self.creators[0].backward(grad)
            self.creators[1].backward(grad)
            
    def __add__(self, other):
        return Tensor(self.data + other.data, creators=[self, other], creation_op="add")
        
    def __repr(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())
    

In [9]:
x = Tensor([1, 2, 3, 4, 5])
y = Tensor([2, 2, 2, 2, 2])

z = x + y
print(z)

z.backward(Tensor(np.array([1, 1, 1, 1, 1])))


[3 4 5 6 7]


In [10]:
print(x.grad)
print(y.grad)
print(z.creators)
print(z.creation_op)

[1 1 1 1 1]
[1 1 1 1 1]
[<__main__.Tensor object at 0x000001BB6DCE4DD8>, <__main__.Tensor object at 0x000001BB6DCE4C88>]
add


In [12]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])
d = Tensor([-1,-2,-3,-4,-5])

e = a+b
f = c+d
g = e+f

g.backward(Tensor(np.array([1,1,1,1,1])))
print(a.grad)

[1 1 1 1 1]


In [13]:
d = a+b
e = b+c
f = d+e

f.backward(Tensor(np.array([1,1,1,1,1])))
print(b.grad.data==np.array([2,2,2,2,2]))

[False False False False False]


In [14]:
print(d)

[3 4 5 6 7]


In [15]:
print(e)

[7 6 5 4 3]


In [16]:
class Tensor(object):
    def __init__(self, data, 
                autograd=False,
                creators=None,
                creation_op=None,
                id=None):
        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
        self.autograd = autograd
        self.children = {}
        
        if(id is None):
            id = np.random.randint(0, 100000)
            
        self.id = id
        
        # 텐서가 얼마나 많은 자식을 가지고 있는지 계속해서 추적
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                    
                else:
                    c.children[self.id] += 1
                    
                    
    # 텐서가 각 자식으로부터 보낸 경사도 개수가 정확한지 확인                
    def all_children_grads_accounted_for(self):
        for id, cnt in self.children.items():
            if(cnt != 0):
                return False
        return True
    
    def backward(self, grad=None, grad_origin=None):
        if(self.autograd):
            if(grad_origin is not None):
                # 역전파가 가능한지 또는 경사도를 기다리고 있는 상태인지를 확인하고, 경사도를 기다리고 있다면 카운터를 감소
                
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1
            
            # 여러 자식의 경사도를 누적       
            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
                
            if(self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    # 실제 역전파 시작
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data, autograd = True, creators=[self, other], creation_op="add")
        
        return Tensor(self.data + other.data)
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())
    
    
        

In [17]:
a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)

d = a+b
e = b+c
f = d+e

In [18]:
f.backward(Tensor(np.array([1,1,1,1,1])))

print(b.grad.data == np.array([2,2,2,2,2]))

[ True  True  True  True  True]


### 자동 미분을 이용해서 신경망 학습하기

In [19]:
np.random.seed(0)

data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
target = np.array([[0], [1], [0], [1]])

In [ ]:
weights_0_1 = np.random.rand(2, 3)
weights_1_2 = np.random.rand(3, 1)

In [21]:
for i in range(10):
    layer_1 = data.dot(weights_0_1)
    layer_2 = layer_1.dot(weights_1_2)
    
    diff = (layer_2 - target)
    sqdiff = (diff)**2
    print("평균제곱오차: ", sqdiff)
    loss = sqdiff.sum(0)
    print("평균제곱오차 손실: ", loss)
    
    layer_1_grad = diff.dot(weights_1_2.T)
    weights_1_2_update = layer_1.T.dot(diff)
    weights_0_1_update = data.T.dot(layer_1_grad)
    
    weights_1_2 -= weights_1_2_update * 0.1
    weights_0_1 -= weights_0_1_update * 0.1
    # 0.1 = learning rate
    
    print("총 손실값: ", loss[0])
    
    

평균제곱오차:  [[0.        ]
 [0.05695952]
 [2.12810023]
 [2.88138025]]
평균제곱오차 손실:  [5.06643999]
총 손실값:  5.066439994622395
평균제곱오차:  [[0.        ]
 [0.2328106 ]
 [0.26230112]
 [0.00087906]]
평균제곱오차 손실:  [0.49599078]
총 손실값:  0.4959907791902342
평균제곱오차:  [[0.        ]
 [0.20128405]
 [0.21650575]
 [0.00027738]]
평균제곱오차 손실:  [0.41806719]
총 손실값:  0.4180671892167177
평균제곱오차:  [[0.        ]
 [0.17018062]
 [0.18258264]
 [0.00021807]]
평균제곱오차 손실:  [0.35298133]
총 손실값:  0.35298133007809646
평균제곱오차:  [[0.        ]
 [0.14186691]
 [0.15509336]
 [0.0002947 ]]
평균제곱오차 손실:  [0.29725496]
총 손실값:  0.2972549636567377
평균제곱오차:  [[0.        ]
 [0.11708024]
 [0.13172121]
 [0.00043116]]
평균제곱오차 손실:  [0.2492326]
총 손실값:  0.2492326038163328
평균제곱오차:  [[0.        ]
 [0.09585954]
 [0.1114102 ]
 [0.00058418]]
평균제곱오차 손실:  [0.20785392]
총 손실값:  0.20785392075862477
평균제곱오차:  [[0.        ]
 [0.07795049]
 [0.09364306]
 [0.00071906]]
평균제곱오차 손실:  [0.17231261]
총 손실값:  0.17231260916265176
평균제곱오차:  [[0.        ]
 [0.06299212]
 [0.07813097]
 [0.

## 자동 추적화 추가하기
* 확률적 경사하강법 최적화기(SGD Optimizer)

In [ ]:
class SGD(object):
    def __inint__(self, parameters, alpha=0.1):
        self.parametes = parameters
        self.alpha = alpha
        
    def zero(self):
        for p in self.parameters:
            p.grad.data *= 0
            
    def step(self, zero=True):
        for p in self.parameters:
            p.data -= p.grad * self.alpha
            
            if(zero):
                p.grad.data *= 0